## Modelo Detección de Anomalías GRUPO EPM
## COVID 19 - MAYO - JULIO 2020

In [5]:
#Se invoca librería para manipulación de base de datos
import pandas as pd
#Se invoca librería para cálculos matriciales
import numpy as np
### Para cálculo del valor entero de un decimal
import math
#Se invoca librería para gráficos
import matplotlib
import matplotlib.pyplot as plt 
#pipeline for ML
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import datasets  
import scipy.io
import scipy.stats as stats
import pickle

In [ ]:
# Celdas de notebook
#%matplotlib inline
#Actualización pip
#!python -m pip install --user --no-warn-script-location --upgrade pip  
#Graficos de Alto nivel
#!python -m pip install plotly 

## Preparación de los datos para el modelo

In [8]:
#Se cargan los datos
dataFrameBase = pd.read_csv("SALIDA_20200831_COVID_HI_V1.csv", delimiter="|" , encoding = "ISO-8859-1")

In [9]:
dataFrameBase.head()

,CEDULA,EDAD,CAMPAMENTO,BLOQUE,HABITACION,CAMPAMENTO_BLOQUE,FRENTES,Cuadrilla,CONTAGIOS,CONTAGIOS_FRENTE,...,LATITUD,LONGITUD,RECOMENDACION,Estado,RECOMENDACION_3,ind_positivo,ind_sospechoso,trasmisibilidad,grado,FALLECIDOS
0,337525,59,VILLA LUZ,1,105,VILLA LUZ1,SIN FRENTE,SIN CUADRILLA,NO,SI,...,7.123447,-75.675293,10.Posible contagio por cuadrilla y frente,SOSPECHOSO,NARANJA,0.0,0.0,0.0,0,NO
1,338602,63,VILLA LUZ,3,111,VILLA LUZ3,SIN FRENTE,SIN CUADRILLA,NO,SI,...,0.000000,0.000000,10.Posible contagio por cuadrilla y frente,SOSPECHOSO,NARANJA,0.0,0.0,0.0,0,NO
2,346551,71,VILLA LUZ,3,-99,VILLA LUZ3,SIN FRENTE,SIN CUADRILLA,NO,SI,...,0.000000,0.000000,16.En casa,SANO,VERDE,0.0,0.0,0.0,0,NO
3,391792,41,TACUI-CUNI,-99,-99,TACUI-CUNI-99,SIN FRENTE,SIN CUADRILLA,NO,SI,...,0.000000,0.000000,10.Posible contagio por cuadrilla y frente,SOSPECHOSO,NARANJA,0.0,0.0,0.0,0,NO
4,403924,55,TACUI-CUNI,3,18,TACUI-CUNI3,SIN FRENTE,SIN CUADRILLA,NO,SI,...,7.065497,-75.684128,10.Posible contagio por cuadrilla y frente,SOSPECHOSO,NARANJA,0.0,0.0,0.0,0,NO


In [10]:
dataFrameBase.columns

Index(['CEDULA', 'EDAD', 'CAMPAMENTO', 'BLOQUE', 'HABITACION',
       'CAMPAMENTO_BLOQUE', 'FRENTES', 'Cuadrilla', 'CONTAGIOS',
       'CONTAGIOS_FRENTE', 'CONTAGIOS_CAMPAMENTO_BLOQUE', 'Rh',
       'Evolucion_Medica', 'Ubicacion_Actual ', 'Nombre_Ubicacion ',
       'Cantidad_Pruebas ', 'Fecha_PRUEBA', 'CONTAGIOS_CUADRILLA',
       'Tos o estornudos frecuentes_NO',
       'Tos o estornudos frecuentes_Count_NO',
       'Tos o estornudos frecuentes_SI',
       'Tos o estornudos frecuentes_Count_SI',
       'Temperatura de 38 grados o más (Fiebre)_NO',
       'Temperatura de 38 grados o más (Fiebre)_Count_NO',
       'Temperatura de 38 grados o más (Fiebre)_SI',
       'Temperatura de 38 grados o más (Fiebre)_Count_SI',
       'Dolor de cabeza y malestar general_NO',
       'Dolor de cabeza y malestar general_Count_NO',
       'Dolor de cabeza y malestar general_SI',
       'Dolor de cabeza y malestar general_Count_SI', 'Mucosidad nasal_NO',
       'Mucosidad nasal_Count_NO', 'Mucosidad 

In [11]:
### Renombrar las columnas
dataFrameBase.rename(columns={"CEDULA":"Cedula"}, inplace=True)
dataFrameBase.rename(columns={"EDAD":"Edad"}, inplace=True)
dataFrameBase.rename(columns={"CAMPAMENTO_BLOQUE":"CampamentoBloque"}, inplace=True)
dataFrameBase.rename(columns={"FRENTES":"Frentes"}, inplace=True)
dataFrameBase.rename(columns={"CONTAGIOS":"Contagios"}, inplace=True)
dataFrameBase.rename(columns={"CONTAGIOS_FRENTE":"ContagiosBloque"}, inplace=True)
dataFrameBase.rename(columns={"CONTAGIOS_CAMPAMENTO_BLOQUE":"ContagiosCampamentoBloque"}, inplace=True)
dataFrameBase.rename(columns={"Rh":"Rh"}, inplace=True)
dataFrameBase.rename(columns={"Tos o estornudos frecuentes_NO":"TosOEstornudosFrecuentesNo"}, inplace=True)
dataFrameBase.rename(columns={"Tos o estornudos frecuentes_Count_NO":"TosOEstornudosFrecuentesCountNO"}, inplace=True)
dataFrameBase.rename(columns={"Tos o estornudos frecuentes_SI":"TosOEstornudosFrecuentesSI"}, inplace=True)
dataFrameBase.rename(columns={"Tos o estornudos frecuentes_Count_SI":"TosOEstornudosFrecuentesCountSi"}, inplace=True)
dataFrameBase.rename(columns={"Temperatura de 38 grados o más (Fiebre)_NO":"TemperaturaDe38GradosOMasFiebreNo"}, inplace=True)
dataFrameBase.rename(columns={"Temperatura de 38 grados o más (Fiebre)_Count_NO":"TemperaturaDe38GradosOMasFiebreCountNo"}, inplace=True)
dataFrameBase.rename(columns={"Temperatura de 38 grados o más (Fiebre)_SI":"TemperaturaDe38GradosOMasFiebreSi"}, inplace=True)
dataFrameBase.rename(columns={"Temperatura de 38 grados o más (Fiebre)_Count_SI":"TemperaturaDe38GradosOMasFiebreCountSi"}, inplace=True)
dataFrameBase.rename(columns={"Dolor de cabeza y malestar general_NO":"DolorDeCabezaYMalestarGeneralNo"}, inplace=True)
dataFrameBase.rename(columns={"Dolor de cabeza y malestar general_Count_NO":"DolorDeCabezayMalestarGeneralCountNo"}, inplace=True)
dataFrameBase.rename(columns={"Dolor de cabeza y malestar general_SI":"DolorDeCabezaYMalestarGeneralSi"}, inplace=True)
dataFrameBase.rename(columns={"Dolor de cabeza y malestar general_Count_SI":"DolorDeCabezaYMalestarGeneralCountSi"}, inplace=True)
dataFrameBase.rename(columns={"Mucosidad nasal_NO":"MucosidadNasalNo"}, inplace=True)
dataFrameBase.rename(columns={"Mucosidad nasal_Count_NO":"MucosidadNasalCountNo"}, inplace=True)
dataFrameBase.rename(columns={"Mucosidad nasal_SI":"MucosidadNasalSi"}, inplace=True)
dataFrameBase.rename(columns={"Mucosidad nasal_Count_SI":"MucosidadNasalCountSi"}, inplace=True)
dataFrameBase.rename(columns={"Dolor de garganta o dificultad al tragar_NO":"DolorDeGargantaODificultadAlTragarNo"}, inplace=True)
dataFrameBase.rename(columns={"Dolor de garganta o dificultad al tragar_Count_NO":"DolorDeGargantaODificultadAlTragarCountNo"}, inplace=True)
dataFrameBase.rename(columns={"Dolor de garganta o dificultad al tragar_SI":"DolorDeGargantaODificultadAlTragarSi"}, inplace=True)
dataFrameBase.rename(columns={"Dolor de garganta o dificultad al tragar_Count_SI":"DolorDeGargantaODificultadAlTragarCountSi"}, inplace=True)
dataFrameBase.rename(columns={"Dificultad para oler o degustar los alimentos_NO":"DificultadParaOlerODegustarLosAlimentosNo"}, inplace=True)
dataFrameBase.rename(columns={"Dificultad para oler o degustar los alimentos_Count_NO":"DificultadParaOlerODegustarLosAlimentosCountNo"}, inplace=True)
dataFrameBase.rename(columns={"Dificultad para oler o degustar los alimentos_SI":"DificultadParaOlerODegustarLosAlimentosSi"}, inplace=True)
dataFrameBase.rename(columns={"Dificultad para oler o degustar los alimentos_Count_SI":"DificultadparaOlerODegustarLoAlimentosCountSi"}, inplace=True)
dataFrameBase.rename(columns={"DIAS_ENCUESTADO_SINTOMAS_SI":"DiasEncuestadoSintomasSi"}, inplace=True)
dataFrameBase.rename(columns={"DIAS_HASTA_HOY_SINTOMAS_SI":"DiasHastaHoySintomasSi"}, inplace=True)
dataFrameBase.rename(columns={"#_SINTOMAS_SI":"NroSintomasSi"}, inplace=True)
dataFrameBase.rename(columns={"RECOMENDACION":"Recomendacion"}, inplace=True)
dataFrameBase.rename(columns={"CAMPAMENTO":"Campamento"}, inplace=True)
dataFrameBase.rename(columns={"BLOQUE":"Bloque"}, inplace=True)
dataFrameBase.rename(columns={"HABITACION":"Habitacion"}, inplace=True)
dataFrameBase.rename(columns={"LATITUD":"Latitud"}, inplace=True)
dataFrameBase.rename(columns={"LONGITUD":"Longitud"}, inplace=True)
dataFrameBase.rename(columns={"Fecha_PRUEBA":"FechaPrueba"}, inplace=True)
dataFrameBase.rename(columns={"Estado":"Estado"}, inplace=True)
dataFrameBase.rename(columns={"Evolucion_Medica":"EvolucionMedica"}, inplace=True)
dataFrameBase.rename(columns={"Ubicacion_Actual ":"UbicacionActual"}, inplace=True)
dataFrameBase.rename(columns={"Nombre_Ubicacion ":"NombreUbicacion"}, inplace=True)
dataFrameBase.rename(columns={"Cantidad_Pruebas ":"CantidadPruebas"}, inplace=True)
dataFrameBase.rename(columns={"RECOMENDACION_3":"Recomendacion3"}, inplace=True)

In [12]:
dataFrameBase.head()

,Cedula,Edad,Campamento,Bloque,Habitacion,CampamentoBloque,Frentes,Cuadrilla,Contagios,ContagiosBloque,...,Latitud,Longitud,Recomendacion,Estado,Recomendacion3,ind_positivo,ind_sospechoso,trasmisibilidad,grado,FALLECIDOS
0,337525,59,VILLA LUZ,1,105,VILLA LUZ1,SIN FRENTE,SIN CUADRILLA,NO,SI,...,7.123447,-75.675293,10.Posible contagio por cuadrilla y frente,SOSPECHOSO,NARANJA,0.0,0.0,0.0,0,NO
1,338602,63,VILLA LUZ,3,111,VILLA LUZ3,SIN FRENTE,SIN CUADRILLA,NO,SI,...,0.000000,0.000000,10.Posible contagio por cuadrilla y frente,SOSPECHOSO,NARANJA,0.0,0.0,0.0,0,NO
2,346551,71,VILLA LUZ,3,-99,VILLA LUZ3,SIN FRENTE,SIN CUADRILLA,NO,SI,...,0.000000,0.000000,16.En casa,SANO,VERDE,0.0,0.0,0.0,0,NO
3,391792,41,TACUI-CUNI,-99,-99,TACUI-CUNI-99,SIN FRENTE,SIN CUADRILLA,NO,SI,...,0.000000,0.000000,10.Posible contagio por cuadrilla y frente,SOSPECHOSO,NARANJA,0.0,0.0,0.0,0,NO
4,403924,55,TACUI-CUNI,3,18,TACUI-CUNI3,SIN FRENTE,SIN CUADRILLA,NO,SI,...,7.065497,-75.684128,10.Posible contagio por cuadrilla y frente,SOSPECHOSO,NARANJA,0.0,0.0,0.0,0,NO


In [13]:
dataFrameBase.columns

Index(['Cedula', 'Edad', 'Campamento', 'Bloque', 'Habitacion',
       'CampamentoBloque', 'Frentes', 'Cuadrilla', 'Contagios',
       'ContagiosBloque', 'ContagiosCampamentoBloque', 'Rh', 'EvolucionMedica',
       'UbicacionActual', 'NombreUbicacion', 'CantidadPruebas', 'FechaPrueba',
       'CONTAGIOS_CUADRILLA', 'TosOEstornudosFrecuentesNo',
       'TosOEstornudosFrecuentesCountNO', 'TosOEstornudosFrecuentesSI',
       'TosOEstornudosFrecuentesCountSi', 'TemperaturaDe38GradosOMasFiebreNo',
       'TemperaturaDe38GradosOMasFiebreCountNo',
       'TemperaturaDe38GradosOMasFiebreSi',
       'TemperaturaDe38GradosOMasFiebreCountSi',
       'DolorDeCabezaYMalestarGeneralNo',
       'DolorDeCabezayMalestarGeneralCountNo',
       'DolorDeCabezaYMalestarGeneralSi',
       'DolorDeCabezaYMalestarGeneralCountSi', 'MucosidadNasalNo',
       'MucosidadNasalCountNo', 'MucosidadNasalSi', 'MucosidadNasalCountSi',
       'DolorDeGargantaODificultadAlTragarNo',
       'DolorDeGargantaODificultadAlTra

In [14]:
#Identificar Columnas que tienen valores nulos
missing_data = dataFrameBase.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("") 

Cedula
False    6576
Name: Cedula, dtype: int64

Edad
False    6576
Name: Edad, dtype: int64

Campamento
False    6576
Name: Campamento, dtype: int64

Bloque
False    6576
Name: Bloque, dtype: int64

Habitacion
False    6576
Name: Habitacion, dtype: int64

CampamentoBloque
False    6576
Name: CampamentoBloque, dtype: int64

Frentes
False    6576
Name: Frentes, dtype: int64

Cuadrilla
False    6576
Name: Cuadrilla, dtype: int64

Contagios
False    6576
Name: Contagios, dtype: int64

ContagiosBloque
False    6576
Name: ContagiosBloque, dtype: int64

ContagiosCampamentoBloque
False    6576
Name: ContagiosCampamentoBloque, dtype: int64

Rh
False    6576
Name: Rh, dtype: int64

EvolucionMedica
False    6576
Name: EvolucionMedica, dtype: int64

UbicacionActual
False    5835
True      741
Name: UbicacionActual, dtype: int64

NombreUbicacion
False    5835
True      741
Name: NombreUbicacion, dtype: int64

CantidadPruebas
False    6576
Name: CantidadPruebas, dtype: int64

FechaPrueba
False    6

In [15]:
dataFrameBase['NroSintomasSi'].describe()

count    6576.000000
mean        0.049878
std         0.305000
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         6.000000
Name: NroSintomasSi, dtype: float64

In [16]:
#Revisar los datos de Evolucion médica
pd.value_counts(dataFrameBase['EvolucionMedica'], sort = True)

SIN_DICTAMEN_MEDICO    5673
POSITIVO-RECUPERADO     902
FALLECIDO                 1
Name: EvolucionMedica, dtype: int64

In [17]:
#Revisar los datos de Recomendación
pd.value_counts(dataFrameBase['Recomendacion'], sort = True)

10.Posible contagio por cuadrilla y frente                                  2853
16.En casa                                                                  2255
18.Recuperado                                                                902
03.Posible contagio por cuadrilla, frente, bloque, campamento y sintomas     146
15.Posible contagio por bloque y campamento                                  117
11.Posible contagio por frente, bloque y campamento                           87
17.Limpio                                                                     86
14.Posible contagio por frente                                                44
06.Posible contagio por cuadrilla, frente y sintomas                          31
02.Posible contagio por más de 3 sintomas                                     17
13.Posible contagio por cuadrilla                                             14
12.Posible contagio por sintomas                                               6
08.Posible contagio por bloq

In [18]:
def calcularClasificacionEpidemiologicaStr(row):
    if row['Recomendacion'] == "01.Contagio":
        valor = "Caso"
    elif row['Recomendacion'] == '02.Posible contagio por más de 3 sintomas':
        valor = "Sospechoso"
    elif row['Recomendacion'] == '03.Posible contagio por cuadrilla, frente, bloque, campamento y sintomas':
        valor = "Sospechoso"
    elif row['Recomendacion'] == '04.Posible contagio por frente, bloque, campamento y sintomas':
        valor = "Sospechoso"
    elif row['Recomendacion'] == '05.Posible contagio por cuadrilla, bloque, campamento y sintomas':
        valor = "Sospechoso"
    elif row['Recomendacion'] == '06.Posible contagio por cuadrilla, frente y sintomas':
        valor = "Sospechoso" 
    elif row['Recomendacion'] == '07.Posible contagio por frente y sintomas':
        valor = "Sospechoso"
    elif row['Recomendacion'] == '08.Posible contagio por bloque, campamento y sintomas':
        valor = "Sospechoso"
    elif row['Recomendacion'] == '12.Posible contagio por sintomas':
        valor = "Sospechoso"
    elif row['Recomendacion'] == '09.Posible contagio por cuadrilla,bloque y campamento':
        valor = "Contacto"  
    elif row['Recomendacion'] == '10.Posible contagio por cuadrilla y frente':
        valor = "Contacto"
    elif row['Recomendacion'] == '11.Posible contagio por frente, bloque y campamento':
        valor = "Contacto"
    elif row['Recomendacion'] == '13.Posible contagio por cuadrilla':
        valor = "Contacto"
    elif row['Recomendacion'] == '14.Posible contagio por frente':
        valor = "Contacto"
    elif row['Recomendacion'] == '15.Posible contagio por bloque y campamento':
        valor = "Contacto"
    elif row['Recomendacion'] == '16.En casa':
        valor = "Sano"
    elif row['Recomendacion'] == '17.Limpio':
        valor = "Sano"   
    elif row['Recomendacion'] == '18.Recuperado':
        valor = "Sano"   
    else:
        valor = "SinInformacion"
    return valor

In [19]:
# Aplicar la función para crear el nuevo atributo epidemiológico
dataFrameBase['ClasificacionEpidemiologica'] = dataFrameBase.apply(calcularClasificacionEpidemiologicaStr, axis=1)

In [20]:
#Observar cómo queda la clasificacion epidemiológica con value counts - NO DEBE HABER SIN INFORMACION
dataFrameBase['ClasificacionEpidemiologica'].value_counts()

Sano          3243
Contacto      3118
Sospechoso     214
Caso             1
Name: ClasificacionEpidemiologica, dtype: int64

#### Etiquetar Anomalías

In [21]:
#Crear función para etiquetar las anomalías y las normales (Anomalía -1 y Normal 1)
def etiquetarAnomalias(row):
    if row['ClasificacionEpidemiologica'] == "Caso":
        valor = -1
    elif row['ClasificacionEpidemiologica'] == 'Sospechoso':
        valor = -1
    elif row['ClasificacionEpidemiologica'] == 'Contacto':
        valor = 1
    elif row['ClasificacionEpidemiologica'] == 'Sano':
        valor = 1
    else:
        valor = 1
    return valor

In [22]:
#Aplicar la función para crear la nueva variable Anomalia
dataFrameBase['Anomalia'] = dataFrameBase.apply(etiquetarAnomalias, axis=1)

In [23]:
#Observar como queda la clasificación de anomalías
dataFrameBase['Anomalia'].value_counts()

 1    6361
-1     215
Name: Anomalia, dtype: int64

#### One-hot encoding variables categóricas

In [24]:
# One-hot encoding variables categóricas
CampamentoBloque_dummies = pd.get_dummies(dataFrameBase.CampamentoBloque)
CampamentoBloque_dummies.head(2)

,HUMAGA-99,HUMAGA1,HUMAGA2,HUMAGA4,SIN CAMPAMENTO ASIGNADO-99,SIN CAMPAMENTO ASIGNADO11,SIN CAMPAMENTO ASIGNADO8,TACUI-CUNI-99,TACUI-CUNI1,TACUI-CUNI10,...,VILLA LUZ39,VILLA LUZ4,VILLA LUZ40,VILLA LUZ41,VILLA LUZ42,VILLA LUZ43,VILLA LUZ5,VILLA LUZ6,VILLA LUZ7,VILLA LUZ9
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
#One-hot encoding variables categóricas
ClasificacionEpidemiologica_dummies = pd.get_dummies(dataFrameBase["ClasificacionEpidemiologica"])
ClasificacionEpidemiologica_dummies.head(2)

,Caso,Contacto,Sano,Sospechoso
0,0,1,0,0
1,0,1,0,0


In [26]:
# One-hot encoding variables categóricas
Rh_dummies = pd.get_dummies(dataFrameBase.Rh)
Rh_dummies.head(2)

,A+,A-,AB+,AB-,B+,B-,O+,O-,XX
0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0


In [27]:
dataFrameBaseIndexIdentificacion = dataFrameBase[['Cedula', 'Anomalia']]

In [28]:
XNumToScale = dataFrameBase[['Edad','TosOEstornudosFrecuentesCountNO','TosOEstornudosFrecuentesCountSi','TemperaturaDe38GradosOMasFiebreCountNo','TemperaturaDe38GradosOMasFiebreCountSi','DolorDeCabezayMalestarGeneralCountNo','DolorDeCabezaYMalestarGeneralCountSi','MucosidadNasalCountNo','MucosidadNasalCountSi','DolorDeGargantaODificultadAlTragarCountNo','DolorDeGargantaODificultadAlTragarCountSi','DificultadParaOlerODegustarLosAlimentosCountNo','DificultadparaOlerODegustarLoAlimentosCountSi', 'DiasEncuestadoSintomasSi','DiasHastaHoySintomasSi','NroSintomasSi', 'CantidadPruebas']]
XNumToScale.head()

,Edad,TosOEstornudosFrecuentesCountNO,TosOEstornudosFrecuentesCountSi,TemperaturaDe38GradosOMasFiebreCountNo,TemperaturaDe38GradosOMasFiebreCountSi,DolorDeCabezayMalestarGeneralCountNo,DolorDeCabezaYMalestarGeneralCountSi,MucosidadNasalCountNo,MucosidadNasalCountSi,DolorDeGargantaODificultadAlTragarCountNo,DolorDeGargantaODificultadAlTragarCountSi,DificultadParaOlerODegustarLosAlimentosCountNo,DificultadparaOlerODegustarLoAlimentosCountSi,DiasEncuestadoSintomasSi,DiasHastaHoySintomasSi,NroSintomasSi,CantidadPruebas
0,59,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,63,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,71,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,41,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,55,29,0,29,0,29,0,29,0,29,0,29,0,0,0,0,1


In [29]:
#Librerías para escalar
from sklearn.preprocessing import MinMaxScaler
scaler = StandardScaler()

In [30]:
#Escalar los datos sin las dummies
XNumScaledSinDummies = scaler.fit_transform(XNumToScale)
XNumScaledSinDummies = pd.DataFrame(XNumScaledSinDummies, columns = ['Edad','TosOEstornudosFrecuentesCountNO','TosOEstornudosFrecuentesCountSi','TemperaturaDe38GradosOMasFiebreCountNo','TemperaturaDe38GradosOMasFiebreCountSi','DolorDeCabezayMalestarGeneralCountNo','DolorDeCabezaYMalestarGeneralCountSi','MucosidadNasalCountNo','MucosidadNasalCountSi','DolorDeGargantaODificultadAlTragarCountNo','DolorDeGargantaODificultadAlTragarCountSi','DificultadParaOlerODegustarLosAlimentosCountNo','DificultadparaOlerODegustarLoAlimentosCountSi', 'DiasEncuestadoSintomasSi','DiasHastaHoySintomasSi','NroSintomasSi','CantidadPruebas'])
XNumScaledSinDummies

,Edad,TosOEstornudosFrecuentesCountNO,TosOEstornudosFrecuentesCountSi,TemperaturaDe38GradosOMasFiebreCountNo,TemperaturaDe38GradosOMasFiebreCountSi,DolorDeCabezayMalestarGeneralCountNo,DolorDeCabezaYMalestarGeneralCountSi,MucosidadNasalCountNo,MucosidadNasalCountSi,DolorDeGargantaODificultadAlTragarCountNo,DolorDeGargantaODificultadAlTragarCountSi,DificultadParaOlerODegustarLosAlimentosCountNo,DificultadparaOlerODegustarLoAlimentosCountSi,DiasEncuestadoSintomasSi,DiasHastaHoySintomasSi,NroSintomasSi,CantidadPruebas
0,1.407509,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,-0.071678,-1.223894,-0.052479,-0.066612,-0.176195,-0.163548,0.117930
1,1.677558,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,-0.071678,-1.223894,-0.052479,-0.066612,-0.176195,-0.163548,0.117930
2,2.217655,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,-0.071678,-1.223894,-0.052479,-0.066612,-0.176195,-0.163548,0.117930
3,0.192291,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,-0.071678,-1.223894,-0.052479,-0.066612,-0.176195,-0.163548,-0.738041
4,1.137461,1.307497,-0.081427,1.300353,-0.030898,1.307524,-0.079717,1.311456,-0.092791,1.303282,-0.071678,1.302479,-0.052479,-0.066612,-0.176195,-0.163548,0.117930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6571,-1.090440,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,-0.071678,-1.223894,-0.052479,-0.066612,-0.176195,-0.163548,-0.738041
6572,-1.090440,0.609939,-0.081427,0.603667,-0.030898,0.610030,-0.079717,0.613880,-0.092791,0.606099,-0.071678,0.605549,-0.052479,-0.066612,-0.176195,-0.163548,-0.738041
6573,-0.550343,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,-0.071678,-1.223894,-0.052479,-0.066612,-0.176195,-0.163548,0.117930
6574,-0.752880,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,-0.071678,-1.223894,-0.052479,-0.066612,-0.176195,-0.163548,-0.738041


In [31]:
DatosTotales = pd.concat([XNumScaledSinDummies, CampamentoBloque_dummies, dataFrameBaseIndexIdentificacion], axis=1)
DatosTotales

,Edad,TosOEstornudosFrecuentesCountNO,TosOEstornudosFrecuentesCountSi,TemperaturaDe38GradosOMasFiebreCountNo,TemperaturaDe38GradosOMasFiebreCountSi,DolorDeCabezayMalestarGeneralCountNo,DolorDeCabezaYMalestarGeneralCountSi,MucosidadNasalCountNo,MucosidadNasalCountSi,DolorDeGargantaODificultadAlTragarCountNo,...,VILLA LUZ40,VILLA LUZ41,VILLA LUZ42,VILLA LUZ43,VILLA LUZ5,VILLA LUZ6,VILLA LUZ7,VILLA LUZ9,Cedula,Anomalia
0,1.407509,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,...,0,0,0,0,0,0,0,0,337525,1
1,1.677558,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,...,0,0,0,0,0,0,0,0,338602,1
2,2.217655,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,...,0,0,0,0,0,0,0,0,346551,1
3,0.192291,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,...,0,0,0,0,0,0,0,0,391792,1
4,1.137461,1.307497,-0.081427,1.300353,-0.030898,1.307524,-0.079717,1.311456,-0.092791,1.303282,...,0,0,0,0,0,0,0,0,403924,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6571,-1.090440,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,...,0,0,0,0,0,0,0,0,1234988195,1
6572,-1.090440,0.609939,-0.081427,0.603667,-0.030898,0.610030,-0.079717,0.613880,-0.092791,0.606099,...,0,0,0,0,0,0,0,0,1234989361,1
6573,-0.550343,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,...,0,0,0,0,0,0,0,0,1235240854,1
6574,-0.752880,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,...,0,0,0,0,0,0,0,0,10372667567,1


In [32]:
y = DatosTotales.pop("Anomalia")
X = DatosTotales

In [33]:
# Dividir en entrenamiento 80% y pruebas 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state= 12345)

In [34]:
X_train.shape

(5260, 87)

In [35]:
X_train.head(2)

,Edad,TosOEstornudosFrecuentesCountNO,TosOEstornudosFrecuentesCountSi,TemperaturaDe38GradosOMasFiebreCountNo,TemperaturaDe38GradosOMasFiebreCountSi,DolorDeCabezayMalestarGeneralCountNo,DolorDeCabezaYMalestarGeneralCountSi,MucosidadNasalCountNo,MucosidadNasalCountSi,DolorDeGargantaODificultadAlTragarCountNo,...,VILLA LUZ4,VILLA LUZ40,VILLA LUZ41,VILLA LUZ42,VILLA LUZ43,VILLA LUZ5,VILLA LUZ6,VILLA LUZ7,VILLA LUZ9,Cedula
5673,-0.752880,1.220303,-0.081427,1.213267,-0.030898,1.220338,-0.079717,1.224259,-0.092791,1.216134,...,0,0,0,0,0,0,0,0,0,1045433152
6428,-0.955416,0.086770,-0.081427,0.081153,-0.030898,0.086908,-0.079717,0.090698,-0.092791,0.083212,...,0,0,0,0,0,0,0,0,0,1143391382


In [36]:
X_test.shape

(1316, 87)

In [37]:
y_train.shape

(5260,)

In [38]:
y_test.shape

(1316,)

## Modelo Supervisado de Detección de Anomalías

## Entrenamiento

### Búsqueda del hiperparámetro de contaminación óptimo

In [39]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
folds = list(skf.split(X_train, y_train))

In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer
from sklearn.covariance import EllipticEnvelope

In [41]:
ell = EllipticEnvelope()
f1sc = make_scorer(f1_score)
## ps = {"contamination": np.linspace(0.01, 0.05, 20)}
ps = {"contamination": np.linspace(0, 0.05, 10)}
search = GridSearchCV(estimator=ell, param_grid=ps, scoring=f1sc, cv=folds)
# Entrenamiento del modelo sin tener en cuenta la cédula
search.fit(X_train[X_train.columns[~X_train.columns.isin(['Cedula'])]], y_train)

C:\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance

C:\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance

C:\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance

GridSearchCV(cv=[(array([ 521,  522,  523, ..., 5257, 5258, 5259]),
                  array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  9...
             estimator=EllipticEnvelope(assume_centered=False,
                                        contamination=0.1, random_state=None,
                                        store_precision=True,
                                        support_fraction=None),
             iid='deprecated', n_jobs=None,
             param_grid={'contamination': array([0.

In [42]:
#Revisar resultados del modelo
df = pd.DataFrame(search.cv_results_)
df[["mean_test_score", "std_test_score", "params"]].sort_values(by=["mean_test_score"], ascending=False).head()

,mean_test_score,std_test_score,params
5,0.988039,0.004742,{'contamination': 0.02777777777777778}
6,0.987398,0.006146,{'contamination': 0.03333333333333333}
3,0.986842,0.004519,{'contamination': 0.016666666666666666}
2,0.986683,0.001717,{'contamination': 0.011111111111111112}
7,0.986658,0.008485,{'contamination': 0.03888888888888889}


In [43]:
# Obtener el modelo óptimo
optimal_ell = search.best_estimator_
optimal_ell

EllipticEnvelope(assume_centered=False, contamination=0.02777777777777778,
                 random_state=None, store_precision=True,
                 support_fraction=None)

### Hacer predicciones con el modelo óptimo usando datos de entrenamiento

In [44]:
# Hacer predicciones con el modelo óptimo usando datos de entrenamiento
pred_train = optimal_ell.predict(X_train[X_train.columns[~X_train.columns.isin(['Cedula'])]])

In [45]:
pred_train_df= pd.DataFrame(pred_train, columns = ['AnomaliaPred'])
pred_train_df.head()

,AnomaliaPred
0,1
1,1
2,1
3,1
4,1


In [46]:
X_test.head()

,Edad,TosOEstornudosFrecuentesCountNO,TosOEstornudosFrecuentesCountSi,TemperaturaDe38GradosOMasFiebreCountNo,TemperaturaDe38GradosOMasFiebreCountSi,DolorDeCabezayMalestarGeneralCountNo,DolorDeCabezaYMalestarGeneralCountSi,MucosidadNasalCountNo,MucosidadNasalCountSi,DolorDeGargantaODificultadAlTragarCountNo,...,VILLA LUZ4,VILLA LUZ40,VILLA LUZ41,VILLA LUZ42,VILLA LUZ43,VILLA LUZ5,VILLA LUZ6,VILLA LUZ7,VILLA LUZ9,Cedula
671,0.462339,0.697134,-0.081427,0.690753,-0.030898,0.697216,-0.079717,0.701077,-0.092791,0.693247,...,0,0,0,0,0,0,0,0,0,11804034
313,-0.010246,0.784329,-0.081427,0.777839,-0.030898,0.784403,-0.079717,0.788274,-0.092791,0.780395,...,0,0,0,0,0,0,0,0,0,7760457
612,1.137461,0.173965,-0.081427,0.168239,-0.030898,0.174095,-0.079717,0.177895,-0.092791,0.170360,...,0,0,0,0,0,0,0,0,0,10546050
4505,-0.685367,0.784329,-0.081427,0.777839,-0.030898,0.784403,-0.079717,0.788274,-0.092791,0.780395,...,0,0,0,0,0,0,0,0,0,1035521247
6107,-0.820392,0.086770,-0.081427,0.081153,-0.030898,0.086908,-0.079717,0.090698,-0.092791,0.083212,...,0,0,0,0,0,0,0,0,0,1095817647


### Hacer predicciones con el modelo óptimo usando datos de pruebas

In [47]:
# Hacer predicciones con el modelo óptimo usando datos de pruebas
optimal_ell = search.best_estimator_
pred_test = optimal_ell.predict(X_test[X_test.columns[~X_test.columns.isin(['Cedula'])]])

In [48]:
pred_test_df= pd.DataFrame(pred_test, columns = ['AnomaliaPred'])
pred_test_df.head()

,AnomaliaPred
0,1
1,1
2,1
3,1
4,1


In [49]:
sum(pred_train == -1), sum(pred_train == 1), sum(pred_test == -1), sum(pred_test == 1)  # Outlier points are predicted as -1

(147, 5113, 37, 1279)

#### Guardar el modelo

In [50]:
import pickle
# save the model to disk
filename = 'modelo_20200831.sav'
pickle.dump(optimal_ell, open(filename, 'wb'))

#### Cargar el modelo

In [51]:
# Después se carga el modelo guardado previamente
# Cargar el modelo
loaded_model = pickle.load(open(filename, 'rb'))

In [52]:
#Verificar la totalidad de los datos
DatosTotales[DatosTotales.columns[~DatosTotales.columns.isin(['Cedula','Anomalia'])]]

,Edad,TosOEstornudosFrecuentesCountNO,TosOEstornudosFrecuentesCountSi,TemperaturaDe38GradosOMasFiebreCountNo,TemperaturaDe38GradosOMasFiebreCountSi,DolorDeCabezayMalestarGeneralCountNo,DolorDeCabezaYMalestarGeneralCountSi,MucosidadNasalCountNo,MucosidadNasalCountSi,DolorDeGargantaODificultadAlTragarCountNo,...,VILLA LUZ39,VILLA LUZ4,VILLA LUZ40,VILLA LUZ41,VILLA LUZ42,VILLA LUZ43,VILLA LUZ5,VILLA LUZ6,VILLA LUZ7,VILLA LUZ9
0,1.407509,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,...,0,0,0,0,0,0,0,0,0,0
1,1.677558,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,...,0,0,0,0,0,0,0,0,0,0
2,2.217655,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,...,0,0,0,0,0,0,0,0,0,0
3,0.192291,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,...,0,0,0,0,0,0,0,0,0,0
4,1.137461,1.307497,-0.081427,1.300353,-0.030898,1.307524,-0.079717,1.311456,-0.092791,1.303282,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6571,-1.090440,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,...,0,0,0,0,0,0,0,0,0,0
6572,-1.090440,0.609939,-0.081427,0.603667,-0.030898,0.610030,-0.079717,0.613880,-0.092791,0.606099,...,0,0,0,0,0,0,0,0,0,0
6573,-0.550343,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,...,0,0,0,0,0,0,0,0,0,0
6574,-0.752880,-1.221151,-0.081427,-1.225133,-0.030898,-1.220894,-0.079717,-1.217258,-0.092791,-1.224006,...,0,0,0,0,0,0,0,0,0,0


## Predicción

#### Hacer predicciones con el modelo cargado

In [53]:
# Hacer predicciones con la totalidad de los datos
DatosTotales['AnomaliaPred'] = loaded_model.predict(DatosTotales[DatosTotales.columns[~DatosTotales.columns.isin(['Cedula','Anomalia'])]])

In [54]:
#Verificar el total de predicciones
sum(DatosTotales['AnomaliaPred'] == -1), sum(DatosTotales['AnomaliaPred'] == 1)

(184, 6392)

In [55]:
AnomaliasPred = DatosTotales[DatosTotales.columns[DatosTotales.columns.isin(['Cedula','AnomaliaPred'])]]

In [56]:
#Generar archivo de anomalías como se requiere
AnomaliasPred.to_csv("AnomaliasPred_20200831.csv", index=False)